In [3]:
%pip install transformers torch torchtext scikit-learn pandas
%pip install torch==2.0.1 torchtext==0.15.2
!pip install tensorflow
%pip install transformers torch scikit-learn
!pip install datasets





Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd 
import torch 
from torch.utils.data import Dataset, DataLoader 
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

# Load your data 
# df = pd.read_csv("final_processed.csv") 
df = pd.read_csv("sampled_data.csv") 

# Drop rows with NaN values
df = df.dropna()

# Display the DataFrame to verify the data 
print(df.head()) 

# Map labels to integers 
label_to_int = {label: i for i, label in enumerate(df['sub_category'].unique())} 
df['sub_category'] = df['sub_category'].map(label_to_int) 

# Set hyperparameters 
MAX_LENGTH = 128 
BATCH_SIZE = 16
EPOCHS = 30
LEARNING_RATE = 5e-5

# Initialize tokenizer 
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define CustomTextDataset 
class CustomTextDataset(Dataset): 
    def __init__(self, dataframe, tokenizer, max_length, text_column, label_column): 
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text_column = text_column
        self.label_column = label_column
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx): 
        # Get text and label for the sample 
        text = self.data.iloc[idx][self.text_column]
        label = self.data.iloc[idx][self.label_column]

        # Tokenize the text
        tokens = self.tokenizer(
            text,
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_tensors="pt"
        ) 

        # Remove the batch dimension from token tensors 
        tokens = {key: val.squeeze(0) for key, val in tokens.items()} 

        # Return tokenized inputs and the label 
        return tokens, torch.tensor(label, dtype=torch.long)

def predict(model, tokenizer, texts, max_length=128):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text in texts:
            tokens = tokenizer( 
                text, 
                padding='max_length', 
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            tokens = {key: val.to(model.device) for key, val in tokens.items()}
            output = model(**tokens)
            _, predicted_class = torch.max(output.logits, dim=1)
            predictions.append(predicted_class.item())
    return predictions

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

unique_labels = df['sub_category'].nunique()
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=unique_labels)

train_dataset = CustomTextDataset(df, tokenizer, MAX_LENGTH, text_column='crimeadditionalinfo', label_column='sub_category')
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}") 
    total_loss = 0 
    for batch in tqdm(train_loader): 
        optimizer.zero_grad()
        tokens, labels = batch
        labels = labels.to(torch.long)  
        outputs = model(**tokens, labels=labels)

        loss = outputs.loss
        loss.backward() 
        optimizer.step() 
        total_loss += loss.item() 

    average_loss = total_loss / len(train_loader) 
    print(f"Average Loss: {average_loss:.4f}") 

# Save the trained model
model.save_pretrained("trained_bert_model")
tokenizer.save_pretrained("trained_bert_tokenizer")

# Predict on unknown texts



g:\CyberGuard AI\cyber_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                              sub_category  \
0                       intimidating email   
1  business email compromiseemail takeover   
2                       hacking/defacement   
3           unauthorised accessdata breach   
4                           email phishing   

                                 crimeadditionalinfo  
0   some unknown user has send an email to my rwa...  
1   millennium semiconductors is into business of...  
2   my bank account was transction, and then i go...  
3   on feb    i installed koko loan reliable rupe...  
4   i met a friend in online game his name is kai...  


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
g:\CyberGuard AI\cyber_venv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/30


100%|██████████| 374/374 [33:29<00:00,  5.37s/it]


...

100%|██████████| 374/374 [20:57<00:00,  3.36s/it]


Average Loss: 0.3006
Epoch 30/30


100%|██████████| 374/374 [20:59<00:00,  3.37s/it]


Average Loss: 0.2584


('trained_bert_tokenizer\\tokenizer_config.json',
 'trained_bert_tokenizer\\special_tokens_map.json',
 'trained_bert_tokenizer\\vocab.txt',
 'trained_bert_tokenizer\\added_tokens.json')

In [ ]:
# unknown_texts = [
 
#     "received URL link for updating KYC from mobile no  after opening received otp and entered and confirmation received saying  KYC done like this received OTP till  KYC after that started receiving sms from bank after  sec delay on amount cut and then stopped further otp Bank is also responsible for delaying sms"
# ]
# predicted_classes = predict(model, tokenizer, unknown_texts)

# int_to_label = {v: k for k, v in label_to_int.items()}

# for text, prediction in zip(unknown_texts, predicted_classes): 
#     predicted_label = int_to_label[prediction]
#     print(f"Text: {text} | Predicted Class: {prediction} | Class Label: {predicted_label}")

NameError: name 'df' is not defined

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd 
import torch 
from torch.utils.data import Dataset, DataLoader 
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

# Load your data 
# df = pd.read_csv("final_processed.csv") 
df = pd.read_csv("sampled_data.csv") 

# Drop rows with NaN values
df = df.dropna()

# Display the DataFrame to verify the data 
print(df.head()) 

# Map labels to integers 
label_to_int = {label: i for i, label in enumerate(df['sub_category'].unique())} 
df['sub_category'] = df['sub_category'].map(label_to_int) 

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("trained_bert_model")
tokenizer = BertTokenizer.from_pretrained("trained_bert_tokenizer")

# Ensure model is in evaluation mode
model.eval()


int_to_label = {v: k for k, v in label_to_int.items()}

# Define the prediction function
def predict(model, tokenizer, texts, max_length=128):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text in texts:
            # Tokenize the input text
            tokens = tokenizer(
                text,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            tokens = {key: val.to(model.device) for key, val in tokens.items()}
            
            # Get model outputs
            output = model(**tokens)
            _, predicted_class = torch.max(output.logits, dim=1)
            predictions.append(predicted_class.item())
    return predictions

# Input texts to classify
unknown_texts = [
    "received URL link for updating KYC from mobile no after opening received otp and entered and confirmation received saying KYC done like this received OTP till KYC after that started receiving sms from bank after sec delay on amount cut and then stopped further otp Bank is also responsible for delaying sms",
    "some unknown user has send an email to my rwa and making my reputation at stake along with my family . i request you to please take a legal action and help me to find the person who has is ruing my personal reputation ."
]

# Perform prediction
predicted_classes = predict(model, tokenizer, unknown_texts)

# Map predictions back to label names
for text, prediction in zip(unknown_texts, predicted_classes):
    predicted_label = int_to_label[prediction]
    print(f"Text: {text}\nPredicted Class: {prediction} | Class Label: {predicted_label}\n")


                              sub_category  \
0                       intimidating email   
1  business email compromiseemail takeover   
2                       hacking/defacement   
3           unauthorised accessdata breach   
4                           email phishing   

                                 crimeadditionalinfo  
0   some unknown user has send an email to my rwa...  
1   millennium semiconductors is into business of...  
2   my bank account was transction, and then i go...  
3   on feb    i installed koko loan reliable rupe...  
4   i met a friend in online game his name is kai...  
Text: received URL link for updating KYC from mobile no after opening received otp and entered and confirmation received saying KYC done like this received OTP till KYC after that started receiving sms from bank after sec delay on amount cut and then stopped further otp Bank is also responsible for delaying sms
Predicted Class: 17 | Class Label: internet banking related fraud

Text: some un

In [11]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("trained_bert_model")
tokenizer = BertTokenizer.from_pretrained("trained_bert_tokenizer")

# Ensure the model is in evaluation mode
model.eval()

# Define the predict function
def predict(model, tokenizer, texts, max_length=128):
    """
    Predict the labels for the given texts using the trained model.
    
    Args:
        model: Trained BERT model.
        tokenizer: Tokenizer used with the BERT model.
        texts: List of input texts to classify.
        max_length: Maximum sequence length for padding and truncation.
    
    Returns:
        List of predicted labels.
    """
    model.to("cuda" if torch.cuda.is_available() else "cpu")
    predictions = []
    with torch.no_grad():
        for text in texts:
            tokens = tokenizer(
                text,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            tokens = {key: val.to(model.device) for key, val in tokens.items()}
            output = model(**tokens)
            _, predicted_class = torch.max(output.logits, dim=1)
            predictions.append(predicted_class.item())
    return predictions

# Mapping integers back to their original labels
int_to_label = {v: k for k, v in label_to_int.items()}

# Example unknown texts for prediction
unknown_texts = [
    "My bank account was hacked, and I lost access to everything. Please help me recover it.",
    "Received a phishing email asking me to update my account details.",
]

# Get predictions
predicted_classes = predict(model, tokenizer, unknown_texts)

# Display predictions with their corresponding labels
for text, prediction in zip(unknown_texts, predicted_classes):
    predicted_label = int_to_label[prediction]
    print(f"Text: {text} | Predicted Class: {prediction} | Class Label: {predicted_label}")


NameError: name 'label_to_int' is not defined

                                category                       sub_category  \
0  Online and Social Media Related Crime  cyber bullying  stalking  sexting   
1                 Online Financial Fraud                  fraud callvishing   
2               Online Gambling  Betting           online gambling  betting   
3  Online and Social Media Related Crime                   online job fraud   
4                 Online Financial Fraud                  fraud callvishing   

                                  crimeaditionalinfo  
0  I had continue received random calls and abusi...  
1  The above fraudster is continuously messaging ...  
2  He is acting like a police and demanding for m...  
3  In apna Job I have applied for job interview f...  
4  I received a call from lady stating that she w...  
Accuracy: 0.9992125984251968

Classification Report:
                                                 precision    recall  f1-score   support

                          Cryptocurrency Crime      